In [1]:
import pandas as pd
import numpy as np

In [ ]:
data0 = pd.read_csv("./training_data/target1.csv")

In [ ]:
data0['time'] = data0['time'].map(lambda x: int(x[-8:-6]))
data0['buy'] = pd.DataFrame({'buy':(data0['generation'] - data0['consumption'])})
data0 = data0.drop(["generation", "consumption"], axis=1)
data0

In [2]:
def generate_data(train):
    X_train, Y_train = [], []
    for i in range(train['time'].shape[0]-8):
        X_train.append(np.array(train['buy'].iloc[i:i+7]))
        Y_train.append(np.array(train['buy'].iloc[i+7:i+8]))
    return np.array(X_train), np.array(Y_train)

In [3]:
# from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector,Input
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model
from keras.models import Sequential

def lstm_model():
    model = Sequential()
    model.add(LSTM(256, input_shape=(7, 1), return_sequences=True))
    model.add(LSTM(256, return_sequences=True))
    model.add(TimeDistributed(Dense(1)))
    model.add(Flatten())
    model.add(Dense(30,activation='linear'))
    model.add(Dense(1,activation='linear'))
    model.compile(loss="mean_absolute_error", optimizer="adam",metrics=['mean_absolute_error', 'accuracy'])
    model.summary()
    return model

Using TensorFlow backend.
/Users/dinghr/miniconda3/envs/T1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/dinghr/miniconda3/envs/T1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/dinghr/miniconda3/envs/T1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/dinghr/miniconda3/e

In [4]:
model = lstm_model()





_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 7, 256)            264192    
_________________________________________________________________
lstm_2 (LSTM)                (None, 7, 256)            525312    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 7, 1)              257       
_________________________________________________________________
flatten_1 (Flatten)          (None, 7)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 30)                240       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 31        
Total params: 790,032
Trainable params: 790,032
Non-trainable params: 0
_________________________________________________________________


In [ ]:
times = [0, 1, 2, 3, 4, 5, 6, 18, 19, 20, 21, 22, 23]
callback = EarlyStopping(monitor="mean_absolute_error", patience=10, verbose=1, mode="auto")
for i in times:
    X_train, Y_train = generate_data(data0[data0['time']==i])
    X_train = X_train.reshape(-1, 7, 1)
    history = model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_split=0.1, callbacks=[callback],shuffle=True)

In [6]:
callback = EarlyStopping(monitor="mean_absolute_error", patience=10, verbose=1, mode="auto")
DATA = [0, 5, 12, 18, 28, 33, 46]
times = [0, 1, 2, 3, 4, 5, 6, 18, 19, 20, 21, 22, 23]
for j in DATA:
    print(f'====================== DATA SOURCE = {j} ======================')
    data0 = pd.read_csv(f"./training_data/target{j}.csv")
    data0['time'] = data0['time'].map(lambda x: int(x[-8:-6]))
    data0['buy'] = pd.DataFrame({'buy':(data0['generation'] - data0['consumption'])})
    data0 = data0.drop(["generation", "consumption"], axis=1)
    for i in times:
        print(f'==================== i = {i} ==================')
        X_train, Y_train = generate_data(data0[data0['time']==i])
        X_train = X_train.reshape(-1, 7, 1)
        history = model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_split=0.1, callbacks=[callback],shuffle=True)

====================== DATA SOURCE = 0 ======================
==================== i = 0 ==================
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 210 samples, validate on 24 samples
Epoch 1/10
210/210 [==============================] - 6s 28ms/step - loss: 0.7004 - mean_absolute_error: 0.7004 - acc: 0.0000e+00 - val_loss: 0.6386 - val_mean_absolute_error: 0.6386 - val_acc: 0.0000e+00
Epoch 2/10
210/210 [==============================] - 0s 2ms/step - loss: 0.4094 - mean_absolute_error: 0.4094 - acc: 0.0000e+00 - val_loss: 0.7791 - val_mean_absolute_error: 0.7791 - val_acc: 0.0000e+00
Epoch 3/10
210/210 [==============================] - 1s 2ms/step - loss: 0.3833 - mean_absolute_error: 0.3833 - acc: 0.0048 - val_loss: 0.4520 - val_mean_absolute_error: 0.4520 - val_acc: 0.0000e+00
Epoch 4/10
210/210 [==============================] - 1s 3ms/step - loss: 0.3626 - mean_absolute_error: 0.3626 - acc: 0.0000e+00 - val_loss: 0

Epoch 6/10
211/211 [==============================] - 1s 3ms/step - loss: 0.6155 - mean_absolute_error: 0.6155 - acc: 0.0000e+00 - val_loss: 1.1368 - val_mean_absolute_error: 1.1368 - val_acc: 0.0000e+00
Epoch 7/10
211/211 [==============================] - 1s 3ms/step - loss: 0.6268 - mean_absolute_error: 0.6268 - acc: 0.0000e+00 - val_loss: 1.0886 - val_mean_absolute_error: 1.0886 - val_acc: 0.0000e+00
Epoch 8/10
211/211 [==============================] - 1s 3ms/step - loss: 0.6202 - mean_absolute_error: 0.6202 - acc: 0.0000e+00 - val_loss: 1.0824 - val_mean_absolute_error: 1.0824 - val_acc: 0.0000e+00
Epoch 9/10
211/211 [==============================] - 1s 3ms/step - loss: 0.6173 - mean_absolute_error: 0.6173 - acc: 0.0000e+00 - val_loss: 1.1065 - val_mean_absolute_error: 1.1065 - val_acc: 0.0000e+00
Epoch 10/10
211/211 [==============================] - 1s 3ms/step - loss: 0.6163 - mean_absolute_error: 0.6163 - acc: 0.0000e+00 - val_loss: 1.0886 - val_mean_absolute_error: 1.0886 -

211/211 [==============================] - 1s 3ms/step - loss: 0.6873 - mean_absolute_error: 0.6873 - acc: 0.0142 - val_loss: 0.6634 - val_mean_absolute_error: 0.6634 - val_acc: 0.0000e+00
Epoch 5/10
211/211 [==============================] - 1s 3ms/step - loss: 0.6985 - mean_absolute_error: 0.6985 - acc: 0.0142 - val_loss: 0.6443 - val_mean_absolute_error: 0.6443 - val_acc: 0.0000e+00
Epoch 6/10
211/211 [==============================] - 1s 3ms/step - loss: 0.7149 - mean_absolute_error: 0.7149 - acc: 0.0142 - val_loss: 0.6776 - val_mean_absolute_error: 0.6776 - val_acc: 0.0000e+00
Epoch 7/10
211/211 [==============================] - 1s 3ms/step - loss: 0.7252 - mean_absolute_error: 0.7252 - acc: 0.0142 - val_loss: 0.7083 - val_mean_absolute_error: 0.7083 - val_acc: 0.0000e+00
Epoch 8/10
211/211 [==============================] - 1s 3ms/step - loss: 0.7378 - mean_absolute_error: 0.7378 - acc: 0.0142 - val_loss: 0.6599 - val_mean_absolute_error: 0.6599 - val_acc: 0.0000e+00
Epoch 9/10


Epoch 3/10
211/211 [==============================] - 1s 3ms/step - loss: 0.4660 - mean_absolute_error: 0.4660 - acc: 0.0047 - val_loss: 0.4601 - val_mean_absolute_error: 0.4601 - val_acc: 0.0000e+00
Epoch 4/10
211/211 [==============================] - 1s 3ms/step - loss: 0.4232 - mean_absolute_error: 0.4232 - acc: 0.0047 - val_loss: 0.4922 - val_mean_absolute_error: 0.4922 - val_acc: 0.0000e+00
Epoch 5/10
211/211 [==============================] - 1s 3ms/step - loss: 0.4336 - mean_absolute_error: 0.4336 - acc: 0.0047 - val_loss: 0.4649 - val_mean_absolute_error: 0.4649 - val_acc: 0.0000e+00
Epoch 6/10
211/211 [==============================] - 1s 3ms/step - loss: 0.4075 - mean_absolute_error: 0.4075 - acc: 0.0047 - val_loss: 0.5171 - val_mean_absolute_error: 0.5171 - val_acc: 0.0000e+00
Epoch 7/10
211/211 [==============================] - 1s 3ms/step - loss: 0.4087 - mean_absolute_error: 0.4087 - acc: 0.0047 - val_loss: 0.4620 - val_mean_absolute_error: 0.4620 - val_acc: 0.0000e+00


210/210 [==============================] - 1s 3ms/step - loss: 0.2704 - mean_absolute_error: 0.2704 - acc: 0.0048 - val_loss: 0.1925 - val_mean_absolute_error: 0.1925 - val_acc: 0.0000e+00
Epoch 2/10
210/210 [==============================] - 1s 3ms/step - loss: 0.2716 - mean_absolute_error: 0.2716 - acc: 0.0048 - val_loss: 0.1826 - val_mean_absolute_error: 0.1826 - val_acc: 0.0000e+00
Epoch 3/10
210/210 [==============================] - 1s 3ms/step - loss: 0.2681 - mean_absolute_error: 0.2681 - acc: 0.0048 - val_loss: 0.1783 - val_mean_absolute_error: 0.1783 - val_acc: 0.0000e+00
Epoch 4/10
210/210 [==============================] - 1s 3ms/step - loss: 0.2664 - mean_absolute_error: 0.2664 - acc: 0.0048 - val_loss: 0.2039 - val_mean_absolute_error: 0.2039 - val_acc: 0.0000e+00
Epoch 5/10
210/210 [==============================] - 1s 3ms/step - loss: 0.2664 - mean_absolute_error: 0.2664 - acc: 0.0048 - val_loss: 0.1680 - val_mean_absolute_error: 0.1680 - val_acc: 0.0000e+00
Epoch 6/10


...

Epoch 3/10
211/211 [==============================] - 1s 3ms/step - loss: 0.2792 - mean_absolute_error: 0.2792 - acc: 0.0000e+00 - val_loss: 0.4011 - val_mean_absolute_error: 0.4011 - val_acc: 0.0000e+00
Epoch 4/10
211/211 [==============================] - 1s 3ms/step - loss: 0.2787 - mean_absolute_error: 0.2787 - acc: 0.0000e+00 - val_loss: 0.4227 - val_mean_absolute_error: 0.4227 - val_acc: 0.0000e+00
Epoch 5/10
211/211 [==============================] - 1s 3ms/step - loss: 0.3018 - mean_absolute_error: 0.3018 - acc: 0.0000e+00 - val_loss: 0.4163 - val_mean_absolute_error: 0.4163 - val_acc: 0.0000e+00
Epoch 6/10
211/211 [==============================] - 1s 3ms/step - loss: 0.2767 - mean_absolute_error: 0.2767 - acc: 0.0000e+00 - val_loss: 0.4046 - val_mean_absolute_error: 0.4046 - val_acc: 0.0000e+00
Epoch 7/10
211/211 [==============================] - 1s 3ms/step - loss: 0.2646 - mean_absolute_error: 0.2646 - acc: 0.0000e+00 - val_loss: 0.4201 - val_mean_absolute_error: 0.4201 - 

Epoch 2/10
211/211 [==============================] - 1s 3ms/step - loss: 0.1247 - mean_absolute_error: 0.1247 - acc: 0.0000e+00 - val_loss: 0.1421 - val_mean_absolute_error: 0.1421 - val_acc: 0.0000e+00
Epoch 3/10
211/211 [==============================] - 1s 3ms/step - loss: 0.1219 - mean_absolute_error: 0.1219 - acc: 0.0000e+00 - val_loss: 0.1640 - val_mean_absolute_error: 0.1640 - val_acc: 0.0000e+00
Epoch 4/10
211/211 [==============================] - 1s 3ms/step - loss: 0.1253 - mean_absolute_error: 0.1253 - acc: 0.0000e+00 - val_loss: 0.1522 - val_mean_absolute_error: 0.1522 - val_acc: 0.0000e+00
Epoch 5/10
211/211 [==============================] - 1s 3ms/step - loss: 0.1228 - mean_absolute_error: 0.1228 - acc: 0.0000e+00 - val_loss: 0.1435 - val_mean_absolute_error: 0.1435 - val_acc: 0.0000e+00
Epoch 6/10
211/211 [==============================] - 1s 3ms/step - loss: 0.1185 - mean_absolute_error: 0.1185 - acc: 0.0000e+00 - val_loss: 0.1467 - val_mean_absolute_error: 0.1467 - 

==================== i = 3 ==================
Train on 211 samples, validate on 24 samples
Epoch 1/10
211/211 [==============================] - 1s 3ms/step - loss: 0.2491 - mean_absolute_error: 0.2491 - acc: 0.0047 - val_loss: 0.3503 - val_mean_absolute_error: 0.3503 - val_acc: 0.0417
Epoch 2/10
211/211 [==============================] - 1s 3ms/step - loss: 0.2468 - mean_absolute_error: 0.2468 - acc: 0.0047 - val_loss: 0.3841 - val_mean_absolute_error: 0.3841 - val_acc: 0.0417
Epoch 3/10
211/211 [==============================] - 1s 3ms/step - loss: 0.2420 - mean_absolute_error: 0.2420 - acc: 0.0047 - val_loss: 0.4092 - val_mean_absolute_error: 0.4092 - val_acc: 0.0417
Epoch 4/10
211/211 [==============================] - 1s 3ms/step - loss: 0.2362 - mean_absolute_error: 0.2362 - acc: 0.0047 - val_loss: 0.3776 - val_mean_absolute_error: 0.3776 - val_acc: 0.0417
Epoch 5/10
211/211 [==============================] - 1s 3ms/step - loss: 0.2333 - mean_absolute_error: 0.2333 - acc: 0.0047 

211/211 [==============================] - 1s 3ms/step - loss: 0.2225 - mean_absolute_error: 0.2225 - acc: 0.0095 - val_loss: 0.3978 - val_mean_absolute_error: 0.3978 - val_acc: 0.0000e+00
==================== i = 18 ==================
Train on 211 samples, validate on 24 samples
Epoch 1/10
211/211 [==============================] - 1s 3ms/step - loss: 0.8480 - mean_absolute_error: 0.8480 - acc: 0.0047 - val_loss: 0.6359 - val_mean_absolute_error: 0.6359 - val_acc: 0.0000e+00
Epoch 2/10
211/211 [==============================] - 1s 3ms/step - loss: 0.7370 - mean_absolute_error: 0.7370 - acc: 0.0047 - val_loss: 1.0049 - val_mean_absolute_error: 1.0049 - val_acc: 0.0000e+00
Epoch 3/10
211/211 [==============================] - 1s 3ms/step - loss: 0.7136 - mean_absolute_error: 0.7136 - acc: 0.0047 - val_loss: 1.0791 - val_mean_absolute_error: 1.0791 - val_acc: 0.0000e+00
Epoch 4/10
211/211 [==============================] - 1s 3ms/step - loss: 0.6773 - mean_absolute_error: 0.6773 - acc: 0

Epoch 9/10
211/211 [==============================] - 1s 3ms/step - loss: 0.3645 - mean_absolute_error: 0.3645 - acc: 0.0000e+00 - val_loss: 0.4406 - val_mean_absolute_error: 0.4406 - val_acc: 0.0000e+00
Epoch 10/10
211/211 [==============================] - 1s 3ms/step - loss: 0.3638 - mean_absolute_error: 0.3638 - acc: 0.0000e+00 - val_loss: 0.4328 - val_mean_absolute_error: 0.4328 - val_acc: 0.0000e+00
==================== i = 22 ==================
Train on 211 samples, validate on 24 samples
Epoch 1/10
211/211 [==============================] - 1s 3ms/step - loss: 0.3081 - mean_absolute_error: 0.3081 - acc: 0.0000e+00 - val_loss: 0.6022 - val_mean_absolute_error: 0.6022 - val_acc: 0.0000e+00
Epoch 2/10
211/211 [==============================] - 1s 3ms/step - loss: 0.3052 - mean_absolute_error: 0.3052 - acc: 0.0000e+00 - val_loss: 0.5898 - val_mean_absolute_error: 0.5898 - val_acc: 0.0000e+00
Epoch 3/10
211/211 [==============================] - 1s 3ms/step - loss: 0.2954 - mean_abs

In [7]:
model.save('night.h5')